Source: https://github.com/vsa-datascience/vlaams-twitter-sentiment-model/tree/master/twitter_sentiment_classifier

In [ ]:
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df_annotations=  pd.read_csv('full_cleaned_dataset.csv')
df_annotations# to display the first 5 lines of loaded data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_annotations['text'], df_annotations['label'], test_size=0.3, random_state=42, stratify=df_annotations['label'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
Train=pd.DataFrame()
for i in X_train.index:
    Train=Train.append(df_annotations.iloc[i])
Test=pd.DataFrame()
for i in X_test.index:
    Test=Test.append(df_annotations.iloc[i])
Val=pd.DataFrame()
for i in X_val.index:
    Val=Val.append(df_annotations.iloc[i])

In [ ]:
import json

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        data = data.to_dict('index')
        for i in data:
            json_record = json.dumps(data[i], ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [ ]:
dump_jsonl(Train, 'train.jsonl')
dump_jsonl(Test, 'test.jsonl')
dump_jsonl(Val, 'val.jsonl')

In [ ]:
with open(os.path.expanduser('test_train.jsonl'),encoding="utf8") as f:
    annotations_train = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_train)} training annotations")

# Testing
with open(os.path.expanduser('test_test.jsonl'), encoding="utf8") as f:
    annotations_test = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_test)} testing annotations")

# Validation
with open(os.path.expanduser('test_val.jsonl'), encoding="utf8") as f:
    annotations_val = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_val)} validation annotations")

In [ ]:
from twitter_sentiment_classifier.sentiment_model import SentimentModel

In [ ]:
model = SentimentModel(
    cuda=False,
)
model

In [ ]:
model.reset(initialise=False)

In [ ]:
# Number of training epochs
N_EPOCHS = 2

# Number of last layers prone to change during training
N_LAYERS = 1

In [ ]:
model.train(
    n_epochs=N_EPOCHS,
    train_samples=annotations_train,
    val_samples=annotations_val,
    test_samples=annotations_test,
    n_last_layers=N_LAYERS,
)

In [ ]:
from twitter_sentiment_classifier.sentiment_model import Encoder

In [ ]:
# Load in the model
encoder = Encoder(
    cuda=False,
)
encoder

In [ ]:
annotations_combined = annotations_train + annotations_val + annotations_test

In [ ]:
encoder.encode_data(
    data_samples=annotations_combined,
)

In [ ]:
from twitter_sentiment_classifier.sentiment_model import Classifier

In [ ]:
classifier = Classifier()
classifier

In [ ]:
# Number of training epochs
N_EPOCHS = 10

In [ ]:
classifier.train(
    n_epochs=N_EPOCHS,
    train_samples=annotations_train,
    val_samples=annotations_val,
    test_samples=annotations_test,
)

In [ ]:
import pickle

In [ ]:
filename = 'BERT_model_selftrained'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename = 'BERT_classifier_selftrained'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.eval(annotations_test)
print(result)